In [ ]:
using LFAToolkit
using LinearAlgebra
using Plots

In [ ]:
# 1D Setup
p = 2
n_fine = 2
dimension = 1
mesh = Mesh1D(1.0)
basis_ctof = TensorH1LagrangeHProlongationBasis(p, 1, dimension, n_fine)

# operators
function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end
# -- coarse
basis = TensorH1LagrangeBasis(p, p + 1, 1, dimension)
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
diffusion_coarse = Operator(diffusionweakform, mesh, inputs, outputs)

# -- fine
basis =
    TensorH1LagrangeMacroBasis(p, p + 1, 1, dimension, n_fine)
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
diffusion_fine = Operator(diffusionweakform, mesh, inputs, outputs)

# preconditioning
chebyshev = Chebyshev(diffusion_fine)
multigrid = PMultigrid(diffusion_fine, diffusion_coarse, chebyshev, [basis_ctof])

In [ ]:
# compute full operator symbols
numberruns = 250
θ_min = -π/2
θ_max = 3π/2
n = [3]
ν = [1, 1]

# compute and plot smoothing factor
# -- compute
maxeigenvalues = zeros(numberruns)
for i in 1:numberruns
    θ = [θ_min + (θ_max - θ_min)*i/numberruns]
    A = computesymbols(multigrid, n, ν, θ)
    eigenvalues = [abs(val) for val in eigvals(A)]
    maxeigenvalues[i] = max(eigenvalues...)
end

# -- plot
xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
plot(
    xrange,
    xlabel="θ/π",
    xtickfont=font(12, "Courier"),
    maxeigenvalues,
    ytickfont=font(12, "Courier"),
    ylabel="spectral radius",
    linewidth=3,
    legend=:none,
    title="1D H-Multigrid Preconditioner Symbol",
    color=palette(:tab10)[1]
)
ylims!(0.0, max(maxeigenvalues...) * 1.1)

In [ ]:
savefig("hmultigridSymbol1D")

In [ ]:
# 2D Setup
p = 2
n_fine = 2
dimension = 2
mesh = Mesh2D(1.0, 1.0)
basis_ctof = TensorH1LagrangeHProlongationBasis(p, 1, dimension, n_fine)

# operators
function diffusionweakform(du::Array{Float64}, w::Array{Float64})
    dv = du*w[1]
    return [dv]
end
# -- coarse
basis = TensorH1LagrangeBasis(p, p + 1, 1, dimension)
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
diffusion_coarse = Operator(diffusionweakform, mesh, inputs, outputs)

# -- fine
basis =
    TensorH1LagrangeMacroBasis(p, p + 1, 1, dimension, n_fine)
inputs = [
    OperatorField(basis, [EvaluationMode.gradient]),
    OperatorField(basis, [EvaluationMode.quadratureweights]),
]
outputs = [OperatorField(basis, [EvaluationMode.gradient])]
diffusion_fine = Operator(diffusionweakform, mesh, inputs, outputs)

# preconditioning
chebyshev = Chebyshev(diffusion_fine)
multigrid = PMultigrid(diffusion_fine, diffusion_coarse, chebyshev, [basis_ctof])

In [ ]:
# compute full operator symbols
numberruns = 250
θ_min = -π/2
θ_max = 3π/2
n = [3]
ν = [1, 1]

# compute and plot smoothing factor
# -- compute
maxeigenvalues = zeros(numberruns, numberruns)
for i in 1:numberruns, j in 1:numberruns
    θ = [
        θ_min + (θ_max - θ_min)*i/numberruns,
        θ_min + (θ_max - θ_min)*j/numberruns
    ]
    A = computesymbols(multigrid, n, ν, θ)
    eigenvalues = [abs(val) for val in eigvals(A)]
    maxeigenvalues[i, j] = max(eigenvalues...)
end

# -- plot
xrange = θ_min/π:(θ_max - θ_min)/π/(numberruns-1):θ_max/π
heatmap(
    xrange,
    xlabel="θ/π",
    xtickfont=font(12, "Courier"),
    xrange,
    ylabel="θ/π",
    maxeigenvalues,
    ytickfont=font(12, "Courier"),
    title="2D H-Multigrid Preconditioner Symbol",
    transpose=true,
    aspect_ratio=:equal,
    color=:cividis
)
xlims!(θ_min/π, θ_max/π)
ylims!(θ_min/π, θ_max/π)

In [ ]:
savefig("hmultigridSymbol2D")